<a href="https://colab.research.google.com/github/P3aar/Cartorioebac/blob/main/2_AgregacoesBasicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark = SparkSession.builder.getOrCreate()
clientes = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pagamentos_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('drive/MyDrive/Planilha excel ANALISTA DE DADOS/pedidos.csv',header=True, inferSchema=True)

In [ ]:
clientes.show(2)
pagamentos_pedido.show(2)
pedidos.show(2)
itens_pedido.show(2)

+--------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-----------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|
+--------------------+--------------------+-----------+--------------------+--------------+
only showing top 2 rows

+--------------------+-------------------+--------------+------------------+---------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|
+--------------------+-------------------+--------------+------------------+---------------+
|b81ef226f3fe1789b...|                  1|   credit_card|                 8|          99.33|
|a9810da82917af2d9...|                  1|   credit

In [ ]:
clientes.agg(countDistinct(col('estado_cliente')).alias('qtd_estado_distintos')).show()

+--------------------+
|qtd_estado_distintos|
+--------------------+
|                  27|
+--------------------+



In [ ]:
pagamentos_pedido.groupBy('tipo_pagamento').agg(
    avg('valor_pagamento').alias('pagamento_medio'),
    var_samp('valor_pagamento').alias('pagamento variancia')
  ).show()

+--------------+------------------+-------------------+
|tipo_pagamento|   pagamento_medio|pagamento variancia|
+--------------+------------------+-------------------+
|        boleto| 145.0344354023452| 45616.869821012544|
|   not_defined|               0.0|                0.0|
|   credit_card|163.31902063935797|  49336.98820414979|
|       voucher| 65.70335411255421| 13344.682202516573|
|    debit_card|142.57017004578165| 60414.395995128165|
+--------------+------------------+-------------------+



In [ ]:
pedidos.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- data_compra_pedido: timestamp (nullable = true)
 |-- data_aprovacao_pedido: timestamp (nullable = true)
 |-- data_envio_transportadora: timestamp (nullable = true)
 |-- data_entrega_cliente: timestamp (nullable = true)
 |-- data_estimada_entrega: timestamp (nullable = true)



In [ ]:
pedidos_ordenados = pedidos.orderBy('data_compra_pedido')

In [ ]:
datas_compras_df = pedidos_ordenados.groupBy('status_pedido').agg(
    first('data_compra_pedido').alias('primeiro'),
    last('data_compra_pedido').alias('ultimo'))
datas_compras_df.show()

+-------------+-------------------+-------------------+
|status_pedido|           primeiro|             ultimo|
+-------------+-------------------+-------------------+
|      shipped|2016-09-04 21:15:19|2018-09-03 09:06:57|
|     canceled|2016-09-05 00:15:34|2018-10-17 17:30:18|
|     approved|2017-02-06 20:18:17|2017-04-25 01:25:34|
|     invoiced|2016-10-04 13:02:10|2018-08-14 18:45:08|
|      created|2017-11-06 13:12:34|2018-02-09 17:21:04|
|    delivered|2016-09-15 12:16:38|2018-08-29 15:00:37|
|  unavailable|2016-10-05 14:16:28|2018-08-21 12:21:00|
|   processing|2016-10-05 22:44:13|2018-07-23 18:03:03|
+-------------+-------------------+-------------------+



In [ ]:
pedidos.withColumn('compra_anual', year('data_compra_pedido')).groupBy('compra_anual').count().orderBy(asc('compra_anual')).show()

+------------+-----+
|compra_anual|count|
+------------+-----+
|        2016|  329|
|        2017|45101|
|        2018|54011|
+------------+-----+



In [ ]:
pedidos.withColumn('compra_anual', year('data_compra_pedido')).groupBy('compra_anual').count().orderBy(desc('compra_anual')).show()

+------------+-----+
|compra_anual|count|
+------------+-----+
|        2018|54011|
|        2017|45101|
|        2016|  329|
+------------+-----+



In [ ]:
itens_pedido.groupBy('id_pedido').agg(
    sum(col('preco') + col('valor_frete')),
    sum(col('preco') + col('valor_frete')).alias('valor_total_pedido'),
    format_number(sum(col('preco') + col('valor_frete')),2).alias('valor_total_pedido')
).show()

+--------------------+--------------------------+------------------+------------------+
|           id_pedido|sum((preco + valor_frete))|valor_total_pedido|valor_total_pedido|
+--------------------+--------------------------+------------------+------------------+
|014405982914c2cde...|         78.42999999999999| 78.42999999999999|             78.43|
|019886de8f385a39b...|                     188.4|             188.4|            188.40|
|01a6ad782455876aa...|                     50.09|             50.09|             50.09|
|01d907b3e209269e1...|        169.76000000000002|169.76000000000002|            169.76|
|028dc52e12ddda803...|                     61.72|             61.72|             61.72|
|036dd381dfb3ec75e...|                     69.14|             69.14|             69.14|
|03ebfa9712b7dbc70...|                     55.78|             55.78|             55.78|
|0420da8d50a378401...|                     74.63|             74.63|             74.63|
|05afef1c185862cab...|          

In [ ]:
spark.stop()